# **Sall-e Project**

In [ ]:
!pip install opencv-python
!pip install pillow-avif-plugin
!pip install ultralytics

# Detect Inference 2
[HF Source](https://huggingface.co/turhancan97/yolov5-obb-trash-detection)\
Acc: High

In [ ]:
import yolov5
import cv2
import os

# Define paths
TESTING_FOLDER = "/content/drive/My Drive/Sall-e/testing"
DETECT_FOLDER = "/content/drive/My Drive/Sall-e/detect2"
MODEL_NAME = 'turhancan97/yolov5-detect-trash-classification'

# Ensure the detect folder exists
os.makedirs(DETECT_FOLDER, exist_ok=True)

# Load the pre-trained YOLOv5 model
model = yolov5.load(MODEL_NAME)

# Set model parameters
model.conf = 0.25  # NMS confidence threshold
model.iou = 0.15   # NMS IoU threshold
model.max_det = 1000  # Maximum number of detections per image

# Process each testing image
for i in range(1, 7):
    image_path = os.path.join(TESTING_FOLDER, f"testing_{i}.jpg")
    detect_path = os.path.join(DETECT_FOLDER, f"detect_{i}.jpg")

    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading {image_path}")
        continue

    # Perform object detection
    results = model(img, size=416)

    # Draw bounding boxes on the image
    for result in results.pred[0]:
        x1, y1, x2, y2, conf, cls = result.tolist()
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Draw the rectangle and confidence label
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"Garbage {conf:.2f}"
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the processed image directly into the detect3 folder
    cv2.imwrite(detect_path, img)
    print(f"Detection completed for {image_path}, saved as {detect_path}")

print("All detections completed and saved in the detect2 folder.")

# Detect Inference 3
[HF Source](https://huggingface.co/Yorai/detr-resnet-50_finetuned_detect-waste)\
Acc: Medium

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
import cv2
import os
from PIL import Image
import numpy as np

# Define paths
TESTING_FOLDER = "/content/drive/My Drive/Sall-e/testing"
DETECT_FOLDER = "/content/drive/My Drive/Sall-e/detect3"
MODEL_PATH = "Yorai/detr-resnet-50_finetuned_detect-waste"

# Ensure the detect folder exists
os.makedirs(DETECT_FOLDER, exist_ok=True)

# Load the DETR model and processor
processor = DetrImageProcessor.from_pretrained(MODEL_PATH)
model = DetrForObjectDetection.from_pretrained(MODEL_PATH)

# Process each testing image
for i in range(1, 7):
    image_path = os.path.join(TESTING_FOLDER, f"testing_{i}.jpg")
    detect_path = os.path.join(DETECT_FOLDER, f"detect_{i}.jpg")

    # Read the image
    image = Image.open(image_path).convert("RGB")
    img_cv = cv2.imread(image_path)
    if img_cv is None:
        print(f"Error loading {image_path}")
        continue

    # Preprocess image for DETR
    inputs = processor(images=image, return_tensors="pt")

    # Perform object detection
    with torch.no_grad():
        outputs = model(**inputs)

    # Process detections
    target_sizes = torch.tensor([image.size[::-1]])  # PIL image size is (width, height)
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)[0]

    # Draw bounding boxes on the image
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        x1, y1, x2, y2 = map(int, box.tolist())  # Get bbox coordinates
        confidence = score.item()  # Get confidence score

        # Draw the rectangle and confidence label
        cv2.rectangle(img_cv, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label_text = f"Garbage {confidence:.2f}"
        cv2.putText(img_cv, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the processed image
    cv2.imwrite(detect_path, img_cv)
    print(f"Detection completed for {image_path}, saved as {detect_path}")

print("All detections completed and saved in the detect3 folder.")